In [2]:
import nltk
import re
import pprint
import random
import json
import pprint
import os
import sys
import glob

class Markov(object):
    def __init__(self, order=2, dictFile="", maxWordInSentence=20, fileEncoding="utf-8"):
        self.table = {}
        self.inputLineCount = 0
        self.inputWordCount = 0
        self.setOrder( order )
        self.setMaxWordInSentence(maxWordInSentence)
        if dictFile:
            self.loadDictionary(dictFile, fileEncoding)


    def setOrder(self, order=2):
        self.order = order

	def loadDictionary(self, dictFile, fileEncoding="utf-8"):
		print("Loaded dictionary file: " + dictFile)
		with open(dictFile, 'r', encoding=fileEncoding) as inf:
			self.table = eval(inf.read())

		# pprint.pprint(self.table)

	def readFile(self, filename, fileEncoding="utf-8"):
		with  open(filename, "r", encoding=fileEncoding) as file:
			strLine = " ".join(file)
			self.processSection(strLine)

	def processSection(self,line ):
		#print('---------------------------------')
		# global lineCount, wordCount, table, keyLen
		sent_text = nltk.sent_tokenize(line) # this gives us a list of sentences
            print(sent_text)
		for sentence in sent_text:
			self.inputLineCount = self.inputLineCount  + 1

			tokens = sentence.split()
			keyList = [ ];

			#Add a special key with just beginning words
			self.table.setdefault( '#BEGIN#', []).append(tokens[0:self.order ]);

			#loop through each word, and if we have enough to add dictionary item, then add
			for item in tokens:
				
				#item = self.__clean(item)

				if len(keyList) < self.order :  #not enough items
					keyList.append(item)
					continue

				#If we already have the item, then add it, otherwise add to empty list
				self.table.setdefault( tuple(keyList), []).append(item)

				#Remove the first word and push last word on to it
				keyList.pop(0)
				keyList.append(item)
				self.inputWordCount = self.inputWordCount + 1

		#print(self.table)
		return

	def setMaxWordInSentence(self, maxWordInSentence):
		self.maxWordInSentence = maxWordInSentence

	def genText(self):	
		key = list(random.choice(self.table['#BEGIN#']))

		genStr = " ".join( key )
		for _ in range( self.maxWordInSentence ):
			#print('---------------------')
			#print('key: ', key)
			#print(tuple(key))
			newKey = self.table.setdefault( tuple(key), None) 
			if(not newKey):
				break

			#print('new key: ', newKey)
			newVal = random.choice( newKey )
			#print('new val: ', newVal)
			genStr = genStr + " " + newVal

			key.pop(0)
			key.append(newVal)

		return genStr

	def getLineCount(self):
		return self.inputLineCount

	def getWordCount(self):
		return self.inputWordCount

	def outputDict(self, filename, fileEncoding="utf-8"):
		with open(filename, 'w', encoding=fileEncoding) as file:
			pprint.pprint(self.table, file)
		#pprint.pprint(self.table,markovDictFile)

	def __clean(self, text):
		text = re.sub('<.+?>', '. ', text)
		text = re.sub('&.+?;', '', text)
		text = re.sub('[\']{1}', '', text)
		text = re.sub('[^a-zA-Z0-9\s_\-\?:;\.,!\(\)\"]+', ' ', text)
		text = re.sub('\s+', ' ', text)
		text = re.sub('(\.\s*)+', '. ', text)
		return text


In [ ]:
train = True
dirPath = os.path.dirname(os.path.realpath("__file__"))
dataPath = os.path.abspath(os.path.join(dirPath, '../../', 'data'))
datasetPath =  os.path.abspath(os.path.join(dataPath, "Newsroom/2_content.txt"))
outputPath = os.path.abspath(os.path.join(dirPath, '../../', 'output'))

print(datasetPath)
keyLen = 3
fileList = []
#dictionaryFilePath = os.path.abspath(os.path.join(outputPath, "data_obama.txt"))
dictionaryFilePath = os.path.abspath(os.path.join(outputPath, "newsroom.txt"))
print(dictionaryFilePath)
if train :
    fileList += glob.glob(datasetPath)
    print("Input files")
    print(fileList)
    markovObj = Markov(keyLen)


    for file in fileList:
        print(file)
        try:
            markovObj.readFile(file, "utf-8")
        except:
            markovObj.readFile(file, "windows-1252")

    markovObj.outputDict(dictionaryFilePath)

    print( "Generated Markov dictionary %s with processing %s input lines and %s input words " % ( dictionaryFilePath, str(markovObj.getLineCount()), str(markovObj.getWordCount()) ) )


In [ ]:
maxWordInSentence = 50
genNSentences = 2
outputText = []


markovObj1 = Markov(dictFile=dictionaryFilePath, maxWordInSentence= maxWordInSentence)

for _ in range( genNSentences ):
    text = markovObj1.genText() 
    print( text )
    if len(text) <= 140 and text.endswith('.'):
        outputText.append(text)

print("\n\n")
print(outputText)